In [ ]:
import collections, math, random, sys
import torch
torch.set_default_device('cuda') # uncomment this line to use GPU
sys.path.append('/kaggle/input/nlphw1')
from utils import *
traindata = read_mono('/kaggle/input/nlphw1/data/large', delim='')
devdata = read_mono('/kaggle/input/nlphw1/data/dev', delim='')
testdata = read_mono('/kaggle/input/nlphw1/data/test', delim='')

vocab = Vocab()
for words in traindata:
    vocab |= words

In [ ]:
logits = torch.normal(mean=0, std=0.01, 
                      size=(len(vocab),), 
                      requires_grad=True)

In [ ]:

class myLSTM(torch.nn.Module):
    def __init__(self,vocab,dim):
        super().__init__()
        self.vocab = vocab
        self.dim = dim
        self.lstm = torch.nn.LSTMCell(len(vocab),dim) #should I pass in hidden_dim
        #self.output = torch.nn.Linear(hidden_dim,len(vocab))
        self.linear = torch.nn.Linear(d,len(vocab))
        self.log_softmax = torch.nn.LogSoftmax(dim=0)

    def start(self):
        hidden_state = torch.zeros(self.dim)
        cell_state = torch.zeros(self.dim)
        return hidden_state, cell_state
    
    def step(self, state, num):
        input_step = torch.nn.functional.one_hot(torch.tensor(num),num_classes = len(self.vocab)).float()
        hidden_state, cell_state = self.lstm(input_step,state)
        y = self.linear(hidden_state)
        y = self.log_softmax(y)
        return (hidden_state,cell_state), y
    
    def forward(self, full_seq):
        hidden_state = torch.zeros(self.dim)
        cell_state = torch.zeros(self.dim)
        output = torch.empty((full_seq.shape[0],len(vocab)))
        for t, x_t in enumerate(full_seq):
            hidden_state, cell_state = self.lstm(x_t, (hidden_state, cell_state))
            y = self.linear(hidden_state)
            y = self.log_softmax(y)
            output[t] = y
        return output

In [ ]:
def validate(model, correct_data):
    total = 0
    correct = 0
    for i, sentance in enumerate(correct_data):
        prediction = []
        q = model.start()
        for j,letter in enumerate(sentance[:-1]): #skip eos
            rnn_letter = model.vocab.numberize(letter)
            q, probs = model.step(q,rnn_letter)
            index = model.vocab.numberize(sentance[j+1])
            prediction = torch.argmax(probs)
            if prediction == index:
                correct += 1
            total += 1
    return correct / total

In [ ]:
d = 512
model = myLSTM(vocab,d)
o = torch.optim.Adam(model.parameters(),lr=2e-3)
prev_score = 0
epochs = 10
for epoch in range(epochs): #do 5, now 10 epochs
    random.shuffle(traindata)
    for i, sentance in enumerate(traindata):
        log_sum = 0
        predict_probs = []
        q = model.start()
        for j, letter in enumerate(sentance[:-1]): #no EOS
            rnn_letter = model.vocab.numberize(letter)
            q, probs = model.step(q, rnn_letter)
            index = model.vocab.numberize(sentance[j+1])
            predict_prob = probs[index]
            predict_probs.append(predict_prob)
        sum_logs = sum(predict_probs)
        loss = sum_logs * -1 #want to maximize not minimize
        o.zero_grad()
        loss.backward()
        o.step()
    print("Finished epoch", epoch)
    cur_score = validate(model,devdata)
    print("Accuracy: {:3f}%".format(cur_score * 100))
    if cur_score < prev_score:
        for k in o.param_groups:
            k['lr'] /=2
    prev_score = cur_score